In [3]:
import pandas as pd
import time
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from IPython.display import display, Image
import re
options = Options()
options.add_argument("--headless=new")
driver = webdriver.Chrome(options=options)
def show_screen(width, height):
    driver.set_window_size(width, height)
    driver.save_screenshot("out.png")
    display(Image("out.png"))
options = Options()
pd.set_option('display.max_rows', 95)

In [4]:
pff_draft = pd.read_csv("PFF NFL draft rankings.txt", delimiter=",")
pff_draft = pff_draft.rename(columns={"Rank":"PFFBigBoardRank"})

In [5]:
url = "https://www.fantasypros.com/nfl/rankings/dynasty-rookies-overall.php"
driver.get(url)
time.sleep(5)
rows = driver.find_elements(By.CSS_SELECTOR,"td.sticky-cell.sticky-cell-two",)
rank = []
name = []
i = 1
for row in rows:
   rank.append(i)
   i += 1
   name.append(row.text.split("(")[0].strip())
fantasyPros = pd.DataFrame({"Fantasy Pros Rank":rank, "Name" : name})
  

In [6]:
fantasyPros_PFF = pd.merge(pff_draft,fantasyPros,on="Name", how="left")


In [7]:
reddit1 = pd.read_csv("Reddit1.csv").rename(columns={"Rank":"Reddit1 Rank"})
reddit1 = reddit1[["Reddit1 Rank","Name"]]
first3 = pd.merge(fantasyPros_PFF,reddit1,on="Name", how="outer")
first3["Fantasy Pros Rank"] = first3["Fantasy Pros Rank"].fillna(100)
first3["Reddit1 Rank"] = first3["Reddit1 Rank"].fillna(40)

In [8]:
url = "https://www.rotoballer.com/early-2024-fantasy-football-rookie-rankings-pre-nfl-draft/1320176"
driver.get(url)
time.sleep(3)
table = driver.find_elements(By.TAG_NAME,"tr")
rank = []
name = []
for row in table:
    try:
        row = row.text.split(" ")
        if row[1] != "Rank":
            rank.append(row[1])
            if len(row) == 5:
                name.append(' '.join(row[2:4]))
            elif len(row) == 6:
                name.append(' '.join(row[2:5]))
    except:
        pass
rotoballer = pd.DataFrame({"Roto Rank": rank, "Name":name})
name_maps = {
'Tyrone Tracy Jr.' : 'Tyrone Tracy',
'BrIan Thomas Jr.' : 'Brian Thomas Jr.',
'A.J. Barner' : 'AJ Barner'
}
for index,row in rotoballer.iterrows():
    if row["Name"] in name_maps:
        rotoballer.at[index, 'Name'] = name_maps[row["Name"]]
first4 = pd.merge(first3,rotoballer,on="Name",how="left")
first4["Roto Rank"] = first4["Roto Rank"].fillna(150)

In [9]:
url = "https://www.footballguys.com/article/2024-dynasty-rookie-average-draft-position-01"
driver.get(url)
time.sleep(3)
rank = []
name = []
rows = driver.find_elements(By.TAG_NAME, "tr")
for row in rows:
    row = row.text.split(" ")
    if row[0] != 'RANK':
        rank.append(row[0])
        if len(row) == 7:
            name.append(' '.join(row[2:5]))
        elif len(row) == 6:
            name.append(' '.join(row[2:4]))
name_maps = {
'Tyrone Tracy Jr.' : 'Tyrone Tracy',
'A.J. Barner' : 'AJ Barner',
'Joe Milton' : 'Joe Milton III',
'Issac Guerendo' : 'Isaac Guerendo',
'Frank Gore Jr' : 'Frank Gore Jr.',
}
fgadp = pd.DataFrame({"FG ADP": rank, "Name":name})
for index, row in fgadp.iterrows():
    if row["Name"] in name_maps:
        fgadp.at[index,"Name"] = name_maps[row["Name"]]
first5 = pd.merge(first4,fgadp,on="Name",how = "left")
first5['FG ADP'] = first5["FG ADP"].fillna(100)

In [19]:
name_maps = {
    'AD Mitchell':'Adonai Mitchell',
    'Jonathan Brooks':'Jonathon Brooks',
    'Braelen Allen':'Braelon Allen',
    'Ja’Lynn Polk':"Ja'Lynn Polk",
    'Ja’Tavion Sanders': "Ja'Tavion Sanders" ,
    'JJ McCarthy':'J.J. McCarthy',
    'Brendan Rice':'Brenden Rice'
}
reddit2 = pd.read_csv("Reddit2.txt",header=None)
reddit2.rename(columns={0: 'Name'}, inplace=True)
reddit2["Reddit 2 Rank"] = list(range(1,41))
reddit2["Name"] = reddit2["Name"].apply(lambda name: name.strip() + '.' if name.endswith('Jr') else name.strip())
reddit2["Name"] = reddit2["Name"].apply(lambda name: name_maps[name] if name in name_maps else name.strip())
first6 = pd.merge(first5,reddit2,on="Name",how = "left")
first6["Reddit 2 Rank"] = first6["Reddit 2 Rank"].fillna(50)
first6["Total Score"] =  first6["FG ADP"].astype(int) + first6["PFFBigBoardRank"].astype(int)/2 + first6["Fantasy Pros Rank"].astype(int) + first6["Reddit1 Rank"].astype(int) +  first6["Reddit 2 Rank"].astype(int) + first6["Roto Rank"].astype(int)
first6 = first6.sort_values(by="Total Score")
first6

,PFFBigBoardRank,Name,Posistion,College,Fantasy Pros Rank,Reddit1 Rank,Roto Rank,FG ADP,Total Score,Reddit 2 Rank
0,2,Marvin Harrison Jr.,WR,OHIO STATE,1.0,2.0,1,2,8.0,1.0
2,4,Malik Nabers,WR,LSU,2.0,5.0,2,4,18.0,3.0
3,6,Rome Odunze,WR,WASHINGTON,3.0,6.0,3,7,24.0,2.0
4,7,Brock Bowers,TE,GEORGIA,4.0,8.0,5,6,31.5,5.0
1,1,Caleb Williams,QB,USC,5.0,1.0,6,1,33.5,20.0
7,25,Brian Thomas Jr.,WR,LSU,6.0,14.0,4,9,49.5,4.0
5,21,Jayden Daniels,QB,LSU,9.0,4.0,10,3,57.5,21.0
9,22,Adonai Mitchell,WR,TEXAS,13.0,11.0,8,12,61.0,6.0
8,29,Troy Franklin,WR,OREGON,7.0,12.0,9,10,65.5,13.0
6,3,Drake Maye,QB,N CAROLINA,14.0,3.0,21,5,73.5,29.0
